# Auto deflate an uploaded zip into a bucket

### Imports

In [ ]:
!pip install boto3 tqdm

In [1]:
import boto3
import json
from tqdm import tqdm_notebook as tqdm
import os

### Constants

In [2]:
profile_name = 'josefigueredo' # or default
region_name ='us-east-1'
origin_bucket = 'test-zip-origin-jf'
arn_origin_bucket = 'arn:aws:s3:::' + origin_bucket
destiny_bucket = 'test-zip-destiny-jf'
arn_destiny_bucket = 'arn:aws:s3:::' + destiny_bucket
role_lambda_s3_name = 'LambdaExecutionFullS3'
arn_aws_lambda_execution_fulls3_role = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
arn_amazon_s3_full_access = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
function_name = 'deflater'
runtime = 'python3.7'
handler = 'lambda_unzip.lambda_handler'
lambda_zip_file_path = './lambda.zip'
lambda_zip_file_name = 'lambda.zip'
aws_lambda_group_name = '/aws/lambda/deflater'

### Clients

In [3]:
boto3.setup_default_session(profile_name = profile_name)

s3_client = boto3.client('s3', region_name = region_name)
iam_client = boto3.client('iam', region_name = region_name)
lambda_client = boto3.client('lambda', region_name = region_name)
logs_client = boto3.client('logs', region_name = region_name)

## Infrastructure

### S3

In [4]:
s3_create_bucket_origin_response = s3_client.create_bucket(Bucket = origin_bucket)
s3_create_bucket_origin_response

{'ResponseMetadata': {'RequestId': 'B3BA1CD568DA3DA3',
  'HostId': 'ueiny+hJB2G34o8EJXwWxml25Pn2q8O0i0opJlHUPy48KbfsTik4VNClg19s1w5dXpDh0O1VeMQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ueiny+hJB2G34o8EJXwWxml25Pn2q8O0i0opJlHUPy48KbfsTik4VNClg19s1w5dXpDh0O1VeMQ=',
   'x-amz-request-id': 'B3BA1CD568DA3DA3',
   'date': 'Mon, 07 Oct 2019 02:25:18 GMT',
   'location': '/test-zip-origin-jf',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/test-zip-origin-jf'}

In [5]:
s3_create_bucket_destiny_response = s3_client.create_bucket(Bucket = destiny_bucket)
s3_create_bucket_destiny_response

{'ResponseMetadata': {'RequestId': '4FFFC186422C5D68',
  'HostId': 'PxLknEQDmszwg+gcPbFhGkmECLkqrJW9MRTkH7EOsE1V/bBZofTdLVRF8+D7F/wapf7M5Arhug0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'PxLknEQDmszwg+gcPbFhGkmECLkqrJW9MRTkH7EOsE1V/bBZofTdLVRF8+D7F/wapf7M5Arhug0=',
   'x-amz-request-id': '4FFFC186422C5D68',
   'date': 'Mon, 07 Oct 2019 02:25:21 GMT',
   'location': '/test-zip-destiny-jf',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/test-zip-destiny-jf'}

### Role

In [6]:
role_policy_document = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
role_policy_document

{'Version': '2012-10-17',
 'Statement': [{'Sid': '',
   'Effect': 'Allow',
   'Principal': {'Service': 'lambda.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

In [7]:
json_role_policy_document = json.dumps(role_policy_document)
json_role_policy_document

'{"Version": "2012-10-17", "Statement": [{"Sid": "", "Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]}'

In [8]:
iam_create_role_response = iam_client.create_role(
  RoleName = role_lambda_s3_name,
  AssumeRolePolicyDocument = json_role_policy_document,
)
iam_create_role_response

{'Role': {'Path': '/',
  'RoleName': 'LambdaExecutionFullS3',
  'RoleId': 'AROAVYUVUBFCHSE4FQTVA',
  'Arn': 'arn:aws:iam::396524521796:role/LambdaExecutionFullS3',
  'CreateDate': datetime.datetime(2019, 10, 7, 2, 25, 30, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Sid': '',
     'Effect': 'Allow',
     'Principal': {'Service': 'lambda.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': 'bb6905ef-e8a9-11e9-8659-cd21a764f458',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bb6905ef-e8a9-11e9-8659-cd21a764f458',
   'content-type': 'text/xml',
   'content-length': '823',
   'date': 'Mon, 07 Oct 2019 02:25:29 GMT'},
  'RetryAttempts': 0}}

In [9]:
arn_role = iam_create_role_response.get('Role').get('Arn')
arn_role

'arn:aws:iam::396524521796:role/LambdaExecutionFullS3'

In [10]:
iam_attach_role_1_response = iam_client.attach_role_policy(
    RoleName = role_lambda_s3_name,
    PolicyArn = arn_aws_lambda_execution_fulls3_role,
)
iam_attach_role_1_response

{'ResponseMetadata': {'RequestId': 'bd0a8158-e8a9-11e9-8659-cd21a764f458',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bd0a8158-e8a9-11e9-8659-cd21a764f458',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 07 Oct 2019 02:25:32 GMT'},
  'RetryAttempts': 0}}

In [11]:
iam_attach_role_2_response = iam_client.attach_role_policy(
    RoleName = role_lambda_s3_name,
    PolicyArn = arn_amazon_s3_full_access
)
iam_attach_role_2_response

{'ResponseMetadata': {'RequestId': 'bdbd56ea-e8a9-11e9-8659-cd21a764f458',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bdbd56ea-e8a9-11e9-8659-cd21a764f458',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 07 Oct 2019 02:25:33 GMT'},
  'RetryAttempts': 0}}

In [12]:
iam_get_role = iam_client.get_role(RoleName = role_lambda_s3_name)
iam_get_role

{'Role': {'Path': '/',
  'RoleName': 'LambdaExecutionFullS3',
  'RoleId': 'AROAVYUVUBFCHSE4FQTVA',
  'Arn': 'arn:aws:iam::396524521796:role/LambdaExecutionFullS3',
  'CreateDate': datetime.datetime(2019, 10, 7, 2, 25, 30, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Sid': '',
     'Effect': 'Allow',
     'Principal': {'Service': 'lambda.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]},
  'MaxSessionDuration': 3600},
 'ResponseMetadata': {'RequestId': 'be94a407-e8a9-11e9-8659-cd21a764f458',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'be94a407-e8a9-11e9-8659-cd21a764f458',
   'content-type': 'text/xml',
   'content-length': '830',
   'date': 'Mon, 07 Oct 2019 02:25:35 GMT'},
  'RetryAttempts': 0}}

### Lambda

In [ ]:
%%bash
rm -r ./pytemp
mkdir ./pytemp
cd pytemp
pip install boto3 --target .
cat << EOF > lambda_unzip.py
from __future__ import print_function

import urllib
from urllib.parse import quote
import zipfile
import boto3
import io

print('Loading function')

s3 = boto3.client('s3')

destiny_bucket = 'test-zip-destiny-jf'

def lambda_handler(event, context):
    key = quote(event['Records'][0]['s3']['object']['key'].encode('utf8'))
    origin_bucket = quote(event['Records'][0]['s3']['object']['key'].encode('utf8'))
    try:
        obj = s3.get_object(
            Bucket = origin_bucket,
            Key = key,
        )
        putObjects = []
        # Write zip file files into destiny_bucket
        with io.BytesIO(obj["Body"].read()) as tf:
            tf.seek(0)
            with zipfile.ZipFile(tf, mode='r') as zipf:
                for file in zipf.infolist():
                    fileName = file.filename
                    putFile = s3.put_object(
                        Bucket = destiny_bucket,
                        Key = fileName,
                        Body = zipf.read(file)
                    )
                    putObjects.append(putFile)
                    print(putFile)

        # Delete zip file from origin_bucket
        if len(putObjects) > 0:
            print('')
            deletedObj = s3.delete_object(
                Bucket = origin_bucket,
                Key = key
            )
            print('deleted file:')
            print(deletedObj)
    except Exception as e:
        print(e)
        raise e
EOF
chmod -R 755 .
zip -r ../lambda.zip .
unzip -l ../lambda.zip

In [13]:
env_variables = dict()

In [14]:
with open(lambda_zip_file_path, 'rb') as f:
    zipped_code = f.read()

In [15]:
lambda_create_func_response = lambda_client.create_function(
    FunctionName = function_name,
    Runtime = runtime,
    Role = iam_get_role['Role']['Arn'],
    Handler = handler,
    Code = dict(ZipFile = zipped_code),
    Timeout = 300,
    Environment = dict(Variables = env_variables),
)
lambda_create_func_response

{'ResponseMetadata': {'RequestId': '136beaf9-bcaa-432a-aef7-c73d5282262a',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 07 Oct 2019 02:26:46 GMT',
   'content-type': 'application/json',
   'content-length': '795',
   'connection': 'keep-alive',
   'x-amzn-requestid': '136beaf9-bcaa-432a-aef7-c73d5282262a'},
  'RetryAttempts': 0},
 'FunctionName': 'deflater',
 'FunctionArn': 'arn:aws:lambda:us-east-1:396524521796:function:deflater',
 'Runtime': 'python3.7',
 'Role': 'arn:aws:iam::396524521796:role/LambdaExecutionFullS3',
 'Handler': 'lambda_unzip.lambda_handler',
 'CodeSize': 8597729,
 'Description': '',
 'Timeout': 300,
 'MemorySize': 128,
 'LastModified': '2019-10-07T02:26:45.998+0000',
 'CodeSha256': 'WWXGp7ZC0NDwY+1PkzLKv3Q1rXYXLzZPmdSzVxZ/BQk=',
 'Version': '$LATEST',
 'Environment': {'Variables': {}},
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '5976a006-a293-4c67-a0a6-2627d9f4074b'}

In [16]:
arn_lambda = lambda_create_func_response.get('FunctionArn')
arn_lambda

'arn:aws:lambda:us-east-1:396524521796:function:deflater'

### Trigger

In [17]:
lambda_add_permission_response = lambda_client.add_permission(
    FunctionName = function_name,
    StatementId = 'sid-deflater',
    Action = 'lambda:InvokeFunction',
    Principal = 's3.amazonaws.com',
    SourceArn = arn_origin_bucket
)
lambda_add_permission_response

{'ResponseMetadata': {'RequestId': '95cef67b-2db8-4269-9d27-9b3434ee6bf1',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 07 Oct 2019 02:29:27 GMT',
   'content-type': 'application/json',
   'content-length': '306',
   'connection': 'keep-alive',
   'x-amzn-requestid': '95cef67b-2db8-4269-9d27-9b3434ee6bf1'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"sid-deflater","Effect":"Allow","Principal":{"Service":"s3.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:396524521796:function:deflater","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:s3:::test-zip-origin-jf"}}}'}

In [18]:
notification_configuration = { 
  "LambdaFunctionConfigurations":[ 
    { 
      "LambdaFunctionArn":arn_lambda,
      "Events":[ 
        "s3:ObjectCreated:*"
      ]
    }
  ]
}
notification_configuration

{'LambdaFunctionConfigurations': [{'LambdaFunctionArn': 'arn:aws:lambda:us-east-1:396524521796:function:deflater',
   'Events': ['s3:ObjectCreated:*']}]}

In [19]:
json_notification_configuration = json.dumps(notification_configuration)
json_notification_configuration

'{"LambdaFunctionConfigurations": [{"LambdaFunctionArn": "arn:aws:lambda:us-east-1:396524521796:function:deflater", "Events": ["s3:ObjectCreated:*"]}]}'

In [20]:
s3_put_notif_response = s3_client.put_bucket_notification_configuration(
    Bucket = origin_bucket,
    NotificationConfiguration = notification_configuration
)
s3_put_notif_response

{'ResponseMetadata': {'RequestId': 'C44370D9FB104DD7',
  'HostId': 'qaqrjjvijuaVLfJGTjXwTpMBJMnYXOK6OT0t7NK/tRLF671Kzx4OvFPBMfLrdzGKECDrGpKlfq4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qaqrjjvijuaVLfJGTjXwTpMBJMnYXOK6OT0t7NK/tRLF671Kzx4OvFPBMfLrdzGKECDrGpKlfq4=',
   'x-amz-request-id': 'C44370D9FB104DD7',
   'date': 'Mon, 07 Oct 2019 02:29:33 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

<hr></hr>

## Testing

### Upload

In [22]:
%%bash
rm -r ./uptemp
mkdir ./uptemp
cd uptemp
cat << EOF > file_1.txt
File #1
EOF
cat << EOF > file_2.txt
File #2
EOF
zip -r ../testdata.zip .
unzip -l ../testdata.zip

  adding: file_1.txt (stored 0%)
  adding: file_2.txt (stored 0%)
Archive:  ../testdata.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        8  2019-10-06 23:29   file_1.txt
        8  2019-10-06 23:29   file_2.txt
---------                     -------
       16                     2 files


In [30]:
def hook(t):
    def inner(bytes_amount):
        t.update(bytes_amount)
    return inner

filesize = float(os.path.getsize('testdata.zip'))

with tqdm(total=filesize, unit='B', unit_scale=True, desc=lambda_zip_file_name) as t:
    s3_upload_response = s3_client.upload_file(
        'testdata.zip',
        origin_bucket,
        'testdata.zip',
        Callback = hook(t)
    )

### Check

In [31]:
obj = s3_client.get_object(
    Bucket = destiny_bucket,
    Key = 'file_1.txt',
)
obj

{'ResponseMetadata': {'RequestId': '9387C8BA4DCAE0DE',
  'HostId': 'nU0nnFqWcxi83Cn7rlTPhhpHjPtEALkbbNaeBuLjWG65NNDhc1crTKOPOJMNmfwJCZ1H/Zg3m00=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'nU0nnFqWcxi83Cn7rlTPhhpHjPtEALkbbNaeBuLjWG65NNDhc1crTKOPOJMNmfwJCZ1H/Zg3m00=',
   'x-amz-request-id': '9387C8BA4DCAE0DE',
   'date': 'Mon, 07 Oct 2019 02:38:27 GMT',
   'last-modified': 'Mon, 07 Oct 2019 02:38:25 GMT',
   'etag': '"691aabc8fd355b2b985334afc0003b7f"',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'content-length': '8',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2019, 10, 7, 2, 38, 25, tzinfo=tzutc()),
 'ContentLength': 8,
 'ETag': '"691aabc8fd355b2b985334afc0003b7f"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f60c43bb5c0>}

In [32]:
body = obj['Body'].read()
body

b'File #1\n'

### Logs

In [33]:
response = logs_client.filter_log_events(
    logGroupName = aws_lambda_group_name,
)
response

{'events': [{'logStreamName': '2019/10/07/[$LATEST]c56d3cc87d6e41c3a18a4c9d1a66a6d1',
   'timestamp': 1570415722834,
   'message': 'Loading function\n',
   'ingestionTime': 1570415722852,
   'eventId': '35021440890687323377397779014855515864568234107964555264'},
  {'logStreamName': '2019/10/07/[$LATEST]c56d3cc87d6e41c3a18a4c9d1a66a6d1',
   'timestamp': 1570415722914,
   'message': 'START RequestId: 39e705d5-3fb7-4bcd-88a3-9625aae08130 Version: $LATEST\n',
   'ingestionTime': 1570415722931,
   'eventId': '35021440892471382993280228866273395484552711872197361664'},
  {'logStreamName': '2019/10/07/[$LATEST]c56d3cc87d6e41c3a18a4c9d1a66a6d1',
   'timestamp': 1570415722915,
   'message': '1\n',
   'ingestionTime': 1570415738043,
   'eventId': '35021440892493683738478759507684464667739930732566413312'},
  {'logStreamName': '2019/10/07/[$LATEST]c56d3cc87d6e41c3a18a4c9d1a66a6d1',
   'timestamp': 1570415722915,
   'message': "{'Records': [{'eventVersion': '2.1', 'eventSource': 'aws:s3', 'awsRegi

<hr></hr>

## In case we need to update the lambda

In [ ]:
with open(lambda_zip_file_path, 'rb') as f:
    zipped_code = f.read()

In [ ]:
lambda_update_response = lambda_client.update_function_code(
    FunctionName = function_name,
    ZipFile = zipped_code
)
lambda_update_response

<hr></hr>

## Clean

In [34]:
lambda_client.delete_function(
  FunctionName = function_name
)

{'ResponseMetadata': {'RequestId': '3df98af3-e655-432b-a83d-f1191c2eca3d',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Mon, 07 Oct 2019 02:41:53 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3df98af3-e655-432b-a83d-f1191c2eca3d'},
  'RetryAttempts': 0}}

In [35]:
iam_client.detach_role_policy(
    PolicyArn = arn_aws_lambda_execution_fulls3_role,
    RoleName = role_lambda_s3_name
)

{'ResponseMetadata': {'RequestId': '06d8a1b3-e8ac-11e9-9340-1dcb31d95f83',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '06d8a1b3-e8ac-11e9-9340-1dcb31d95f83',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 07 Oct 2019 02:41:55 GMT'},
  'RetryAttempts': 0}}

In [36]:
iam_client.detach_role_policy(
    PolicyArn = arn_amazon_s3_full_access,
    RoleName = role_lambda_s3_name
)

{'ResponseMetadata': {'RequestId': '07a84d7e-e8ac-11e9-9340-1dcb31d95f83',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '07a84d7e-e8ac-11e9-9340-1dcb31d95f83',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 07 Oct 2019 02:41:56 GMT'},
  'RetryAttempts': 0}}

In [37]:
iam_client.delete_role(
    RoleName = role_lambda_s3_name
)

{'ResponseMetadata': {'RequestId': '08a041b6-e8ac-11e9-9340-1dcb31d95f83',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '08a041b6-e8ac-11e9-9340-1dcb31d95f83',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Mon, 07 Oct 2019 02:41:58 GMT'},
  'RetryAttempts': 0}}

In [38]:
s3_client.delete_bucket(
    Bucket = origin_bucket
)

{'ResponseMetadata': {'RequestId': '8ADFDAC6812B2EA0',
  'HostId': 'RkZKllHIcalWaH7Pq4gAa07O9pNKCFhbzpgcfjZTTY1XhGVlPngpwzA9icOttau+PpxHZGEyUsI=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'RkZKllHIcalWaH7Pq4gAa07O9pNKCFhbzpgcfjZTTY1XhGVlPngpwzA9icOttau+PpxHZGEyUsI=',
   'x-amz-request-id': '8ADFDAC6812B2EA0',
   'date': 'Mon, 07 Oct 2019 02:42:01 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [39]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(destiny_bucket)
bucket.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'E21E470F82CF3EC7',
   'HostId': 'cTBZbVbUwyJr1Vp/VcOYrVtiQFRtnRgKW4TQxN3X/k317mYQFZ/SHOlFtpnnVWvxG6IUguuSAn0=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'cTBZbVbUwyJr1Vp/VcOYrVtiQFRtnRgKW4TQxN3X/k317mYQFZ/SHOlFtpnnVWvxG6IUguuSAn0=',
    'x-amz-request-id': 'E21E470F82CF3EC7',
    'date': 'Mon, 07 Oct 2019 02:42:04 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'file_2.txt'}, {'Key': 'file_1.txt'}]}]

In [40]:
s3_client.delete_bucket(
    Bucket = destiny_bucket
)

{'ResponseMetadata': {'RequestId': 'E2F5615CFC3195D2',
  'HostId': 'sY1gVT/yfZSgyDtuXM43reovj7coUuAx+W2noGEJrza63Fvrx9GUsm22abU5s7/t+43xCqnUmQQ=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'sY1gVT/yfZSgyDtuXM43reovj7coUuAx+W2noGEJrza63Fvrx9GUsm22abU5s7/t+43xCqnUmQQ=',
   'x-amz-request-id': 'E2F5615CFC3195D2',
   'date': 'Mon, 07 Oct 2019 02:42:05 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [41]:
logs_client.delete_log_group(
    logGroupName = aws_lambda_group_name
)

{'ResponseMetadata': {'RequestId': '1631d3bd-c77e-483a-9a54-5967e6b480a5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1631d3bd-c77e-483a-9a54-5967e6b480a5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 07 Oct 2019 02:42:07 GMT'},
  'RetryAttempts': 0}}